In [ ]:
import os
import numpy as np
from pathlib import Path
import torch
from torch import nn


: 

In [ ]:
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

bindata = {}
for fp in sorted(Path("data/ManipNetBIN").iterdir()):
    # print(fp)
    bindata[fp.stem] = np.fromfile(fp)
for k, v in bindata.items():
    print(k, v.shape)

os.chdir("data/ManipNetBIN")
print(os.getcwd())
# num = np.fromfile('Decoder_b0.bin')
# print(num)

: 

In [ ]:
class resblock():
    def __init__(self, in_vec, out_vec):
        self.FC = nn.Linear(in_vec, out_vec)
    def __call__(self, H_zero):
        H_zero = nn.ReLU(H_zero)
        H_one = self.FC(H_zero) + H_zero
        H_one  = nn.ReLU(H_one)
        H_two = self.res_fc(H_one) + H_one + H_zero
        return H_two

: 

In [ ]:
# 訓練に際して、可能であればGPU（cuda）を設定します。GPUが搭載されていない場合はCPUを使用します
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # self.Decoder_b0 = nn.Linear(1536, 220)
        # self.flatten = nn.Flatten()
        # self.pose_fc = nn.Linear(198, 512)
        # self.traj_fc = nn.Linear(336, 512)
        # self.sensor_fc = nn.Linear(1822, 512)
        # self.res_fc = nn.Linear(1536, 1536)
        # self.res1 = resblock(1536, 1536)
        # self.res2 = resblock(1536, 1536)
        self.Decoder_b0 = nn.Linear(1536, 107)
        self.flatten = nn.Flatten()
        self.pose_fc = nn.Linear(192, 256)
        self.traj_fc = nn.Linear(336, 256)
        self.sensor_fc = nn.Linear(1822, 256)
        self.res_fc = nn.Linear(1536, 1536)
        self.res1 = resblock(1536, 1536)
        self.res2 = resblock(1536, 1536)

    def forward(self, pose, traj, sensor):
        x = self.flatten(x)
        pose = self.pose_fc(pose)
        traj = self.traj_fc(traj)
        sensor = self.sensor_fc(sensor)
        out = torch.cat(pose, traj, sensor, dim=1)
        out = self.res1(out)
        out = self.res2(out)
        out = self.decoder_fc(out)
        return out

model = NeuralNetwork().to(device)
print(model)

: 

In [ ]:
for name, param in model.named_parameters():
    print(name, param.shape)

: 

In [ ]:
for k, v in bindata.items():
    print(k, v.shape)

: 

In [ ]:
with torch.no_grad():
    for name, param in model.named_parameters():
        for k, v in bindata.items():
            # print(name, k)
            if name == "Decoder_b0.weight":
                rephape_weight = torch.reshape(torch.from_numpy(bindata["Decoder_w0"]), [107, -1])
                param.copy_(rephape_weight)
            if name == "Decoder_b0.bias":
                rephape_weight = torch.reshape(torch.from_numpy(bindata["Decoder_b0"]), [107])
                param.copy_(rephape_weight)
            if name == "pose_fc.weight":
                rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder0_w0"]), [256, 192])
                param.copy_(rephape_weight)
            if name == "pose_fc.bias":
                rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder0_b0"]), [256])
                param.copy_(rephape_weight)
            if name == "traj_fc.weight":
                rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder1_w0"]), [256, 336])
                param.copy_(rephape_weight)
            if name == "traj_fc.bias":
                rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder1_b0"]), [256])
                param.copy_(rephape_weight)
            if name == "sensor_fc.weight":
                rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder2_w0"]), [256, 1822])
                param.copy_(rephape_weight)
            if name == "sensor_fc.bias":
                rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder2_b0"]), [256])
                param.copy_(rephape_weight)
            #resblock ???
                

: 

In [ ]:
print(model.res2.FC.parameters)

: 